# I. Importing necessary libraries

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import plot_model

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

plt.style.use('default')

# II. HyperParameters And Directories



In [ ]:
TRAIN_DIR = '../input/fer2013/train/'
TEST_DIR = '../input/fer2013/test/'
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.0001

# III. Exploring the Dataset


In [ ]:
def visualize_emotion_counts(directory):
    emotion_classes = os.listdir(directory)
    emotion_counts = []

    for emotion in emotion_classes:
        folder = os.path.join(directory, emotion)
        emotion_counts.append(len(os.listdir(folder)))

    sns.barplot(x=emotion_classes, y=emotion_counts, palette="Purples")

In [ ]:
visualize_emotion_counts(TRAIN_DIR)

In [ ]:
visualize_emotion_counts(TEST_DIR)

# Common Functions

In [ ]:
def get_classes_weights(train_data):
    classes = np.array(train_data.classes)
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(classes), y=classes)
    return dict(enumerate(class_weights))

In [ ]:
def get_predictions(model, test_data):
    return np.argmax(model.predict(test_data), axis=-1)

In [ ]:
def evaluate_model(model,test_data):
    test_data_evaluation=model.evaluate(test_data)
    print("Test Accuracy: {:.2f}%".format(test_data_evaluation[1] * 100))
    print("Test Loss: {:.5f}".format(test_data_evaluation[0]))

In [ ]:
def choose_random_image(test_data, batch_size):
    random_batch = np.random.randint(0, len(test_data))
    random_img_index = np.random.randint(0, batch_size)

    random_img = test_data[random_batch][0][random_img_index]
    true_label = np.argmax(test_data[random_batch][1][random_img_index])  

    return random_img, true_label

# Common Visualization Functions

In [ ]:
def visualize_performance_curves(history):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]
    epochs = range(len(history.history["loss"]))

    plt.figure(figsize=(15,5))

    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label = "training_accuracy")
    plt.plot(epochs, val_accuracy, label = "validation_accuracy")
    plt.title("Accuracy")
    plt.xlabel("epochs")
    plt.grid(True)
    plt.legend()

    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label = "training_loss")
    plt.plot(epochs, val_loss, label = "validation_loss")
    plt.title("Loss")
    plt.xlabel("epochs")
    plt.grid(True)

    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def overview_training_statistics(model, test_data, history):
    evaluate_model(model, test_data)
    visualize_performance_curves(history)

In [ ]:
def visualize_predictions(test_data, batch_size, model, number_images=10):
    emotion_classes=['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

    random_batch = np.random.randint(0, len(test_data) - 1)
    random_img_index = np.random.randint(0, batch_size - 1 , number_images)

    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(12, 8), subplot_kw={'xticks': [], 'yticks': []})

    for i, ax in enumerate(axes.flat):
        random_img = test_data[random_batch][0][random_img_index[i]]
        random_img_label = np.argmax(test_data[random_batch][1][random_img_index[i]])

        model_prediction = np.argmax(model.predict( tf.expand_dims(random_img, axis=0) , verbose=0))
        ax.imshow(random_img)

        color = "green" if emotion_classes[random_img_label] == emotion_classes[model_prediction] else "red"
        ax.set_title(f"True: {emotion_classes[random_img_label]}\nPredicted: {emotion_classes[model_prediction]}", color=color)

    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_predictions_prep(test_data, batch_size, model, number_images=10):
    emotion_classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

    random_batch = np.random.randint(0, len(test_data) - 1)
    random_img_index = np.random.randint(0, batch_size - 1, number_images)

    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(12, 8), subplot_kw={'xticks': [], 'yticks': []})

    for i, ax in enumerate(axes.flat):
        random_img = test_data[random_batch][0][random_img_index[i]]
        img_for_display = (random_img + 1) * 127.5
        img_for_display = img_for_display.astype(np.uint8)
        random_img_label = np.argmax(test_data[random_batch][1][random_img_index[i]])

        model_prediction = np.argmax(model.predict(tf.expand_dims(random_img, axis=0), verbose=0))
        ax.imshow(img_for_display)

        color = "green" if emotion_classes[random_img_label] == emotion_classes[model_prediction] else "red"
        ax.set_title(f"True: {emotion_classes[random_img_label]}\nPredicted: {emotion_classes[model_prediction]}", color=color)

    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_confusion_matrix(true_labels, predicted_labels, model_name):
    emotion_classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
    fig, ax= plt.subplots(figsize=(12,8))
    cm=confusion_matrix(true_labels, predicted_labels)
    sns.heatmap(cm, annot=True, fmt='g',cmap="Purples", ax=ax, xticklabels=emotion_classes, yticklabels=emotion_classes, annot_kws={"size": 18} )
    ax.set_xlabel('Predicted labels',fontsize=16, fontweight='bold')
    ax.set_ylabel('True labels', fontsize=16, fontweight='bold')
    ax.set_title(f'{model_name} Confusion Matrix', fontsize=18, fontweight='bold')
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)

In [ ]:
def visualize_emotion_prediction_stats(true_labels, predicted_labels, model_name):
    emotion_classes=['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
    emotion_counts = []
    cm = confusion_matrix(true_labels, predicted_labels)
    correct_predictions = np.diag(cm)

    for emotion in emotion_classes:
        emotion_counts.append(np.sum(true_labels == emotion_classes.index(emotion)))

    fig, ax = plt.subplots(figsize=(10, 6))
    bar_width = 0.35
    index = np.arange(len(emotion_classes))
    opacity = 0.8

    rects1 = plt.bar(index, emotion_counts, bar_width, alpha=opacity, color='#a6a6f1', label='Total')
    rects2 = plt.bar(index + bar_width, correct_predictions, bar_width, alpha=opacity, color='#a6f1cc', label='Correct')

    plt.xlabel('Emotions', fontsize=16, fontweight='bold')
    plt.ylabel('Counts', fontsize=16, fontweight='bold')
    plt.title(f'Emotion Counts and Correct Predictions ({model_name})', fontsize=16, fontweight='bold')
    plt.xticks(index + bar_width / 2, emotion_classes)
    plt.legend(fontsize=14)

    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
def overview_dataset_statistics(model, test_unshuffled_data, name):
    cnn_predictions = get_predictions(model,test_unshuffled_data)

    true_labels=test_unshuffled_data.labels
    print(classification_report(true_labels, cnn_predictions))

    visualize_confusion_matrix(true_labels, cnn_predictions,name)
    visualize_emotion_prediction_stats(true_labels, cnn_predictions, name)

In [ ]:
def visualize_prediction_with_ensemble(image, true_label, models, class_names):
    if len(models) == 0:
        raise ValueError("Lista cu modele nu poate fi goală.")

    predictions = []
    for model in models:
        prediction = model.predict(tf.expand_dims(image, axis=0), verbose=0)
        predictions.append(prediction)

    mean_prediction = np.mean(predictions, axis=0) 
    predicted_class_index = np.argmax(mean_prediction)  
    plt.imshow(image)
    if predicted_class_index == true_label:
        color = "green"
    else:
        color = "red"

    plt.title(f"True: {class_names[true_label]} - Predicted: {class_names[predicted_class_index]} ({mean_prediction[0][predicted_class_index]:.2f})", color=color)

    plt.axis('off')  
    plt.show()

# IV. Pre-processing Data


* # versiune fara preprocessed input

In [ ]:
train_data_generator = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        zoom_range=0.1,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        validation_split=0.2,
        fill_mode='nearest'
    )

test_data_generator = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
    )

* # versiune preprocessed input

In [ ]:
train_data_generator = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    validation_split=0.2,
)

test_data_generator = ImageDataGenerator(
    validation_split=0.2,
)
#NO RESCALE

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

preprocessing_function = tf.keras.applications.resnet_v2.preprocess_input
train_data_generator = ImageDataGenerator(
        validation_split=0.2,
        preprocessing_function=preprocessing_function
    )

test_data_generator = ImageDataGenerator(
        validation_split=0.2,
        preprocessing_function=preprocessing_function
    )

**common for both preprocess input or without**

In [ ]:
train_data = train_data_generator.flow_from_directory(directory = TRAIN_DIR,
                                                      target_size = (IMG_SIZE,IMG_SIZE),
                                                      batch_size = BATCH_SIZE,
                                                      color_mode = "rgb",
                                                      class_mode = "categorical",
                                                      subset = "training")
validation_data = test_data_generator.flow_from_directory(directory = TRAIN_DIR,
                                                         target_size = (IMG_SIZE ,IMG_SIZE),
                                                         batch_size = BATCH_SIZE,
                                                         color_mode = "rgb",
                                                         class_mode = "categorical",
                                                         subset = "validation")
test_data = test_data_generator.flow_from_directory(directory = TEST_DIR,
                                                    target_size = (IMG_SIZE,IMG_SIZE),
                                                    batch_size = BATCH_SIZE,
                                                    color_mode = "rgb",
                                                    class_mode = "categorical")

In [ ]:
test_unshuffled_data = test_data_generator.flow_from_directory(TEST_DIR,
                                                               target_size=(IMG_SIZE, IMG_SIZE),
                                                               batch_size=BATCH_SIZE,
                                                               color_mode="rgb",
                                                               class_mode="categorical",
                                                               shuffle=False)
true_labels = test_unshuffled_data.classes

In [ ]:
def build_resnet_model(input_shape):
    base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=input_shape)
    
    model = Sequential([
        base_model,
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(7, activation='softmax')
    ])
    
    model.compile(optimizer=Adamax(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
def build_resnet_model_v2(input_shape):
    base_model = tf.keras.applications.ResNet50V2(include_top= False,weights='imagenet', input_shape=(224,224, 3), pooling='avg',classes=7 )
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential([
                      base_model,
                      layers.Flatten(),
                      layers.Dense(512, activation='relu'),
                      layers.Dense(7,activation='softmax')
                    ])
    
    model.summary()
    model.compile(optimizer=Adamax(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])
  
    return model

# **ResNet50V2 Architecture**

# 4.1
* BATCH_SIZE=32
* IMAGE_SIZE=224
* EPOCS=20
* STEPS_PER_EPOCH=717 //MAX
* VALIDATION_STEPS=179 //MAX
* WEIGHTS: YES
* CALLBACKS: NO
* PREPROCESS INPUT FUNCTION: NO

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = 224
EPOCHS = 20
input_shape = (IMG_SIZE, IMG_SIZE, 3)
steps_per_epoch = train_data.n // train_data.batch_size
validation_steps = validation_data.n // test_data.batch_size
class_weights_dict=get_classes_weights(train_data)

In [ ]:
print(steps_per_epoch,validation_steps)

In [ ]:
print(class_weights_dict)

In [ ]:
resnet_model_1 = build_resnet_model(input_shape)

## 2| Model training <a class="anchor" id="train"></a>

In [ ]:
resnet_train_history_1 = resnet_model_1.fit(
    train_data, 
    epochs=EPOCHS, 
    steps_per_epoch=steps_per_epoch, 
    validation_data=validation_data, 
    validation_steps=validation_steps, 
    class_weight=class_weights_dict
)

## 3| Model Evaluation <a class="anchor" id="eval"></a>

In [ ]:
evaluate_model(resnet_model_1, test_data)

## 4| Visualize Model Performance<a class="anchor" id="visualize"></a>

In [ ]:
visualize_performance_curves(resnet_train_history_1)

In [ ]:
visualize_predictions(test_data, BATCH_SIZE, resnet_model_1)

In [ ]:
overview_dataset_statistics(resnet_model_1, test_unshuffled_data, "Resnet50V2_4.1")

# 4.2
* BATCH_SIZE=32
* IMAGE_SIZE=224
* EPOCS=20
* **STEPS_PER_EPOCH=420** //MAX 718
* **VALIDATION_STEPS=110** //MAX 179
* WEIGHTS: YES
* CALLBACKS: NO
* PREPROCESS INPUT FUNCTION: NO


In [ ]:
resnet_model_2=build_resnet_model(input_shape)

In [ ]:
resnet_train_history_2 = resnet_model_2.fit(
    train_data,
    steps_per_epoch=420,
    epochs=20,
    validation_data=validation_data,
    validation_steps=110, 
    class_weight=class_weights_dict)

In [ ]:
overview_training_statistics(resnet_model_2, test_data, resnet_train_history_2)

In [ ]:
visualize_predictions(test_data, BATCH_SIZE, resnet_model_2)

In [ ]:
overview_dataset_statistics(resnet_model_2, test_unshuffled_data, "Resnet50V2_4.2")

# 4.3
* BATCH_SIZE=32
* IMAGE_SIZE=224
* EPOCS=10
* STEPS_PER_EPOCH=718 //MAX
* VALIDATION_STEPS=179 //MAX
* WEIGHTS: NO
* CALLBACKS: NO
* **PREPROCESS INPUT FUNCTION: YES**

In [ ]:
resnet_model_3=build_resnet_model(input_shape)

In [ ]:
resnet_train_history_3 = resnet_model_3.fit(train_data, epochs=10, validation_data=validation_data)

In [ ]:
overview_training_statistics(resnet_model_3, test_data, resnet_train_history_3)

In [ ]:
visualize_predictions_prep(test_data, BATCH_SIZE, resnet_model_3)

In [ ]:
overview_dataset_statistics(resnet_model_3, test_unshuffled_data, "Resnet50V2_4.3")

# 4.4 
* BATCH_SIZE=32
* IMAGE_SIZE=224
* EPOCS=20
* STEPS_PER_EPOCH=718
* VALIDATION_STEPS=179
* WEIGHTS: NO
* **CALLBACKS: YES**
* PREPROCESS INPUT FUNCTION: NO

In [ ]:
resnet_model_4 = build_resnet_model_v2(input_shape)

In [ ]:
resnet_model_4 = build_resnet_model(input_shape)

In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 7, restore_best_weights = True, verbose=1)

reducing_lr_callback = tf.keras.callbacks.ReduceLROnPlateau( monitor='val_accuracy',
                                                  factor=0.2,
                                                  patience=8,
                                                  min_lr=0.000005,
                                                  verbose=1)

callbacks = [early_stopping_callback, reducing_lr_callback]

steps_per_epoch = train_data.n // train_data.batch_size
validation_steps = validation_data.n // validation_data.batch_size

In [ ]:
resnet_history_4 = resnet_model_4.fit(train_data ,validation_data = validation_data ,validation_steps=validation_steps, steps_per_epoch=steps_per_epoch, epochs=100, batch_size=BATCH_SIZE, callbacks = callbacks)

In [ ]:
overview_training_statistics(resnet_model_4, test_data, resnet_history_4)

In [ ]:
visualize_predictions_prep(test_data, BATCH_SIZE, resnet_model_4)

In [ ]:
overview_dataset_statistics(resnet_model_4, test_unshuffled_data, "Resnet50V2_4.4")

* # COMBINED MODELS PREDICTION

In [ ]:
image, true_label = choose_random_image(test_data, BATCH_SIZE)

models = [resnet_model_1, resnet_model_2, resnet_model_3, resnet_model_4]

class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

visualize_prediction_with_ensemble(image, true_label, models, class_names)

#  BATCH_SIZE = 64 

In [ ]:
BATCH_SIZE = 64
IMG_SIZE = 224
EPOCHS = 20
input_shape = (IMG_SIZE, IMG_SIZE, 3)

In [ ]:
train_data = train_data_generator.flow_from_directory(directory = TRAIN_DIR,
                                                    target_size = (IMG_SIZE,IMG_SIZE),
                                                    batch_size = BATCH_SIZE,
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    subset = "training",
                                                   )
validation_data = test_data_generator.flow_from_directory(directory = TRAIN_DIR,
                                                         target_size = (IMG_SIZE ,IMG_SIZE),
                                                         batch_size = BATCH_SIZE,
                                                         color_mode = "rgb",
                                                         class_mode = "categorical",
                                                         subset = "validation",
                                                        )

test_data = test_data_generator.flow_from_directory(directory = TEST_DIR,
                                                   target_size = (IMG_SIZE,IMG_SIZE),
                                                    batch_size = BATCH_SIZE,
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                  )

In [ ]:
test_unshuffled_data = test_data_generator.flow_from_directory(
    TEST_DIR,
    class_mode="categorical",
    target_size=(IMG_SIZE,IMG_SIZE),
    color_mode="rgb",
    shuffle=False,
    batch_size=64
)

In [ ]:
steps_per_epoch = train_data.n // train_data.batch_size
validation_steps = validation_data.n // test_data.batch_size
class_weights_dict=get_classes_weights(train_data)

In [ ]:
print(steps_per_epoch,validation_steps)

# 4.5
* **BATCH_SIZE=64**
* IMAGE_SIZE=224
* EPOCS=20
* **STEPS_PER_EPOCH=300** //MAX=359
* VALIDATION_STEPS=89 //MAX
* WEIGHTS: YES
* CALLBACKS: NO
* PREPROCESS INPUT FUNCTION: NO

In [ ]:
resnet_model_5 = build_resnet_model(input_shape)

In [ ]:
resnet_train_history_5 = resnet_model_5.fit(
    train_data,
    steps_per_epoch=300, 
    epochs=20,
    validation_data=validation_data,
    validation_steps=validation_steps,
    class_weight=class_weights_dict
)

In [ ]:
overview_training_statistics(resnet_model_5, test_data, resnet_train_history_5)

In [ ]:
visualize_predictions(test_data, BATCH_SIZE, resnet_model_5)

In [ ]:
overview_dataset_statistics(resnet_model_5, test_unshuffled_data, "Resnet50V2_4.5")

# 4.6
* BATCH_SIZE=64
* IMAGE_SIZE=224
* **EPOCS=100**
* STEPS_PER_EPOCH=300
* VALIDATION_STEPS=89
* WEIGHTS: YES
* CALLBACKS: NO
* PREPROCESS INPUT FUNCTION: NO

In [ ]:
resnet_model_6 = build_resnet_model(input_shape)

In [ ]:
resnet_train_history_6 = resnet_model_6.fit(
    train_data,
    steps_per_epoch=300,
    epochs=100,
    validation_data=validation_data,
    class_weight=class_weights_dict
)

In [ ]:
overview_training_statistics(resnet_model_6, test_data, resnet_train_history_6)

In [ ]:
visualize_predictions(test_data, BATCH_SIZE, resnet_model_6)

In [ ]:
overview_dataset_statistics(resnet_model_6, test_unshuffled_data, "Resnet50V2_4.6")

# 4.7
* BATCH_SIZE=64
* IMAGE_SIZE=224
* **EPOCS=10**
* STEPS_PER_EPOCH=359 //MAX
* VALIDATION_STEPS=89 //MAX
* WEIGHTS: NO
* CALLBACKS: NO
* **PREPROCESS INPUT FUNCTION: YES**

In [ ]:
resnet_model_7 = build_resnet_model(input_shape)

In [ ]:
resnet_train_history_7 = resnet_model_7.fit(
    train_data,
    epochs=10,
    validation_data=validation_data
)

In [ ]:
overview_training_statistics(resnet_model_7, test_data, resnet_train_history_7)

In [ ]:
visualize_predictions_prep(test_data, BATCH_SIZE, resnet_model_7)

In [ ]:
overview_dataset_statistics(resnet_model_7, test_unshuffled_data, "Resnet50V2_4.7")

# 4.8
* BATCH_SIZE=64
* IMAGE_SIZE=224
* EPOCS=20
* STEPS_PER_EPOCH=359 //MAX
* VALIDATION_STEPS=89 //MAX
* WEIGHTS: NO
* **CALLBACKS: YES**
* **PREPROCESS INPUT FUNCTION: YES**

In [ ]:
resnet_model_8 = build_resnet_model_v2(input_shape)

In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 7, restore_best_weights = True, verbose=1)

reducing_lr_callback = tf.keras.callbacks.ReduceLROnPlateau( monitor='val_accuracy',
                                                  factor=0.2,
                                                  patience=,
                                                  min_lr=0.000005,
                                                  verbose=1)

callbacks = [early_stopping_callback, reducing_lr_callback]
steps_per_epoch = train_data.n // train_data.batch_size
validation_steps = validation_data.n // validation_data.batch_size

In [ ]:
resnet_history_8 = resnet_model_8.fit(train_data, epochs=20, steps_per_epoch=steps_per_epoch, validation_data = validation_data ,validation_steps=validation_steps, batch_size=BATCH_SIZE, callbacks = callbacks)

In [ ]:
overview_training_statistics(resnet_model_8, test_data, resnet_history_8)

In [ ]:
visualize_predictions_prep(test_data, BATCH_SIZE, resnet_model_8)

In [ ]:
overview_dataset_statistics(resnet_model_8, test_unshuffled_data, "Resnet50V2_4.8")

* # COMBINED MODELS PREDICTION

In [ ]:
image, true_label = choose_random_image(test_data, BATCH_SIZE)

models = [resnet_model_5, resnet_model_6, resnet_model_7, resnet_model_8]

class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

visualize_prediction_with_ensemble(image, true_label, models, class_names)